In [1]:
#Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd

In [2]:
# First URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
    
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [3]:
 #Retrieve latest news' title and paragraph; store into variables
results = soup.find('div', class_='image_and_description_container')
    
news_title = results.find_all('img')
news_title = news_title[1]['alt']
news_p = results.find('div', class_='rollover_description_inner').text
news_p = news_p.replace('\n','')

In [4]:
#Set up Chrome.exe
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
#Connect to URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

#Prepare to use Beautiful Soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
#Push FULL IMAGE button to retrieve the image URL
browser.click_link_by_partial_text('FULL IMAGE')

In [7]:
#Retrieve image URL
results = soup.find('a', class_='button fancybox')
feature_image_url = results['data-fancybox-href']
feature_image_url = feature_image_url.replace('medium','large')
feature_image_url = feature_image_url.replace('ip','hires')
url_short = url.rsplit('/spaceimages',1)[0]
feature_image_url = url_short + feature_image_url

In [49]:
#Now let's retrieve Mars weather
url = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')


In [61]:
#Find all the tags that contain tweets
results = soup.find_all('div', class_='content')

for result in results:
    texto = result.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
    x= texto.find("InSight sol")
    
    if x == 0:
        mars_weather = texto.rsplit('pic.twitter',1)[0]
        break


InSight sol 92 (2019-03-01) low -94.4ºC (-137.9ºF) high -12.9ºC (8.8ºF)
winds from the SW at 4.6 m/s (10.2 mph) gusting to 10.4 m/s (23.2 mph)
pressure at 7.20 hPa


In [36]:
#Go for the FACTS table!
url = 'https://space-facts.com/mars/'

In [37]:
#Start retrieving the data from the table
table = pd.read_html(url)

In [38]:
#Organize pandas df
df = table[0]
df.columns = ['Description', 'Value']
df.set_index('Description', inplace=True)

In [39]:
#Transform to HTML string
html_table = df.to_html()

In [40]:
#Set up Chrome.exe
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [41]:
#Connect to URL to find photos of Mars Hemispheres
url = 'https://astrogeology.usgs.gov/maps/mars-viking-hemisphere-point-perspectives'
browser.visit(url)

In [42]:
Hemispheres = ['valles_marineris', 'syrtis_major', 'schiaparelli', 'cerberus']
hemisphere_image_urls = []

for Hemisphere in Hemispheres:
    
    try:
        browser.click_link_by_partial_href(Hemisphere + '_enhanced' )
        #Prepare to use Beautiful Soup
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        title = soup.find('h2', class_='title').text
        #title = title.rsplit(' Enhanced',1)[0]
        image = soup.find('img', class_='wide-image')
        image_link = 'https://astrogeology.usgs.gov' + image['src']
        d = {'title': title,'image_url' : image_link}
        hemisphere_image_urls.append(d)
        
    except:
        browser.find_link_by_text('2').first.click()
        browser.click_link_by_partial_href(Hemisphere + '_enhanced' )
        #Prepare to use Beautiful Soup
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        title = soup.find('h2', class_='title').text
        #title = title.rsplit(' Enhanced',1)[0]
        image = soup.find('img', class_='wide-image')
        image_link = 'https://astrogeology.usgs.gov' + image['src']
        d = {'title': title,'image_url' : image_link}
        hemisphere_image_urls.append(d)

In [62]:
results_dict = {'news_title' : news_title, 'news_p' : news_p, 'feature_image_url' : feature_image_url,\
                    'mars_weather' : mars_weather, 'html_table' : html_table, 'hemisphere_image_urls' : hemisphere_image_urls}

results_dict

{'feature_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA08097_hires.jpg',
 'hemisphere_image_urls': [{'image_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
   'title': 'Valles Marineris Hemisphere Enhanced'},
  {'image_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
   'title': 'Syrtis Major Hemisphere Enhanced'},
  {'image_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
  {'image_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'}],
 'html_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>